# 下載 USD/JPY 外匯數據

這個 notebook 用於下載 USD/JPY 的外匯數據。我們使用 `requests` 庫從 GitHub 倉庫下載 CSV 文件。

數據來源：https://github.com/alleyex/c_forex

In [1]:
# 導入 requests 庫用於發送 HTTP 請求
import requests

# 設置 GitHub 原始文件的 URL
url = "https://raw.githubusercontent.com/alleyex/c_forex/refs/heads/main/playground/usdjpy-m1.csv"

# 發送 GET 請求獲取文件內容
response = requests.get(url)

# 將文件內容寫入本地文件
# 'wb' 模式用於寫入二進制數據
with open("usdjpy-m1.csv", "wb") as f:
    f.write(response.content)

# 文件已成功下載到當前目錄

In [ ]:
# 導入必要的庫
import requests
import os

# 設置文件名和 URL
filename = "usdjpy-m1.csv"
url = "https://raw.githubusercontent.com/alleyex/c_forex/refs/heads/main/playground/usdjpy-m1.csv"

# 檢查文件是否已存在
if os.path.exists(filename):
    print(f"文件 '{filename}' 已存在，跳過下載。")
else:
    # 發送 GET 請求獲取文件內容
    response = requests.get(url)
    
    # 將文件內容寫入本地文件
    with open(filename, "wb") as f:
        f.write(response.content)
    
    print(f"文件 '{filename}' 已成功下載到當前目錄。")

文件 'usdjpy-m1.csv' 已存在，跳過下載。


In [14]:
def download_file(filename, url):
    """
    通用文件下載函數
    
    參數:
        filename (str): 要保存的文件名
        url (str): 文件的 URL
        
    返回:
        bool: 下載是否成功
    """
    import requests
    import os
    
    # 檢查文件是否已存在
    if os.path.exists(filename):
        print(f"文件 '{filename}' 已存在，跳過下載。")
        return True
    
    try:
        # 發送 GET 請求獲取文件內容
        response = requests.get(url)
        response.raise_for_status()  # 檢查請求是否成功
        
        # 將文件內容寫入本地文件
        with open(filename, "wb") as f:
            f.write(response.content)
        
        print(f"文件 '{filename}' 已成功下載到當前目錄。")
        return True
    except Exception as e:
        print(f"下載文件時發生錯誤：{str(e)}")
        return False
    
 # 下載 USD/JPY 數據文件
usdjpy_filename = "usdjpy-m1.csv"
usdjpy_url = "https://raw.githubusercontent.com/alleyex/c_forex/refs/heads/main/playground/usdjpy-m1.csv"
download_file(usdjpy_filename, usdjpy_url)

# 下載 forex_utils.py 文件
forex_utils_filename = "forex_utils.py"
forex_utils_url = "https://raw.githubusercontent.com/alleyex/c_forex/refs/heads/main/playground/forex_utils.py"
download_file(forex_utils_filename, forex_utils_url)

文件 'usdjpy-m1.csv' 已成功下載到當前目錄。
文件 'forex_utils.py' 已成功下載到當前目錄。


True

In [15]:
# 導入 pandas 庫
import pandas as pd

# 讀取 CSV 文件
df = pd.read_csv('usdjpy-m1.csv')

# 將時間列轉換為日期時間格式
df['time'] = pd.to_datetime(df['time'], unit='s')

# 顯示前幾行數據
print("數據前 5 行：")
print(df.head())

# 顯示數據基本信息
print("\n數據基本信息：")
print(df.info())

# 顯示數據統計摘要
print("\n數據統計摘要：")
print(df.describe())

數據前 5 行：
                 time     open     high      low    close  tick_volume  \
0 2024-05-24 15:22:00  157.084  157.088  157.071  157.072         36.0   
1 2024-05-24 15:23:00  157.075  157.076  157.063  157.064         26.0   
2 2024-05-24 15:24:00  157.065  157.078  157.065  157.075         23.0   
3 2024-05-24 15:25:00  157.076  157.100  157.075  157.090         28.0   
4 2024-05-24 15:26:00  157.094  157.100  157.079  157.079         21.0   

   spread  real_volume  
0       3          0.0  
1       3          0.0  
2       4          0.0  
3       2          0.0  
4       3          0.0  

數據基本信息：
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100529 entries, 0 to 100528
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   time         100529 non-null  datetime64[ns]
 1   open         100529 non-null  float64       
 2   high         100529 non-null  float64       
 3   low          100529 no

In [ ]:
from FeatureEngineering import FeatureEngineering
from datetime import datetime, timedelta

import tensorflow as tf
import numpy as np

tool = FeatureEngineering()

# 設置參數
window_size = 8
test_size = 100

indi_df = tool.create_indicators(raw_data)
scal_df = tool.feature_scaling(indi_df)
feat_df, feature_size = tool.create_features(scal_df)
wind_df = tool.windowed(feat_df, window_size)

 # 分割測試數據和訓練數據
test_data = wind_df.tail(test_size)
train_data = wind_df.head(-test_size)

X_test = test_data.iloc[:, :window_size].tail(test_size)
y_test = test_data.y.tail(test_size).to_numpy()

# 打亂訓練數據
# train_data = train_data.sample(frac=1, random_state=2).reset_index(drop=True)
train_data = train_data.sample(frac=1).reset_index(drop=True)

# 提取訓練數據的特徵和標籤
X_train = train_data.iloc[:, :window_size]
y_train = train_data.iloc[:, -1:].to_numpy()

X_train = tool.process_data(X_train, window_size, feature_size)
X_test = tool.process_data(X_test, window_size, feature_size)